# Coffea-Casa Benchmark Example 1

In [31]:
import numpy as np
import pytest
%matplotlib inline
from coffea import hist
import coffea.processor as processor
import awkward as ak

from dask.distributed import Client, LocalCluster
import time
import os
import ipytest

ipytest.config(rewrite_asserts=True, magics=True)
__file__ = "test_coffea_processor_adlexample1.ipynb"

In [32]:
#client = Client("tls://localhost:8786")
cluster = LocalCluster()
client = Client(cluster) 

fileset = {'SingleMu' : ["root://eospublic.cern.ch//eos/root-eos/benchmark/Run2012B_SingleMu.root"]}

/Users/oshadura/CERN_sources/distributed/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 51516 instead
  warnings.warn(
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny

In [33]:
# This program plots an event-level variable (in this case, MET, but switching it is as easy as a dict-key change). It also demonstrates an easy use of the book-keeping cutflow tool, to keep track of the number of events processed.

# The processor class bundles our data analysis together while giving us some helpful tools.  It also leaves looping and chunks to the framework instead of us.
class Processor(processor.ProcessorABC):
    def __init__(self):
        # Bins and categories for the histogram are defined here. For format, see https://coffeateam.github.io/coffea/stubs/coffea.hist.hist_tools.Hist.html && https://coffeateam.github.io/coffea/stubs/coffea.hist.hist_tools.Bin.html
        dataset_axis = hist.Cat("dataset", "")
        MET_axis = hist.Bin("MET", "MET [GeV]", 50, 0, 100)
        
        # The accumulator keeps our data chunks together for histogramming. It also gives us cutflow, which can be used to keep track of data.
        self._accumulator = processor.dict_accumulator({
            'MET': hist.Hist("Counts", dataset_axis, MET_axis),
            'cutflow': processor.defaultdict_accumulator(int)
        })
    
    @property
    def accumulator(self):
        return self._accumulator
    
    def process(self, events):
        output = self.accumulator.identity()
        
        # This is where we do our actual analysis. The dataset has columns similar to the TTree's; events.columns can tell you them, or events.[object].columns for deeper depth.
        dataset = events.metadata["dataset"]
        MET = events.MET.pt
        
        # We can define a new key for cutflow (in this case 'all events'). Then we can put values into it. We need += because it's per-chunk (demonstrated below)
        output['cutflow']['all events'] += ak.size(MET)
        output['cutflow']['number of chunks'] += 1
        
        # This fills our histogram once our data is collected. The hist key ('MET=') will be defined in the bin in __init__.
        output['MET'].fill(dataset=dataset, MET=MET)
        return output

    def postprocess(self, accumulator):
        return accumulator

In [34]:
def coffea_processor_adlexample1(chunk_size):
  output = processor.run_uproot_job(fileset,
                                  treename = 'Events',
                                  processor_instance = Processor(),
                                  executor = processor.dask_executor,
                                  chunksize = chunk_size,
                                  executor_args = {'schema': processor.NanoAODSchema, 'client': client}
                                )

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (<ipython-input-34-91618eb23bad>, line 7)

In [28]:
# Generates a 1D histogram from the data output to the 'MET' key. fill_opts are optional, to fill the graph (default is a line).
def coffea_plot_adlexample1():
    hist.plot1d(output['MET'], overlay='dataset', fill_opts={'edgecolor': (0,0,0,0.3), 'alpha': 0.8})

In [29]:
  @pytest.mark.parametrize("chunk_size", range(100000, 500000, 100000))
  def test_coffea_processor_adlexample1(benchmark, chunk_size):
        benchmark(coffea_processor_adlexample1, chunk_size)
        coffea_plot_adlexample1()

In [30]:
ipytest.run("-qq")

FFFF                                                                     [100%]
=================================== FAILURES ===================================
__________________ test_coffea_processor_adlexample1[100000] ___________________

benchmark = <pytest_benchmark.fixture.BenchmarkFixture object at 0x7fdca03a4a90>
chunk_size = 100000

    @pytest.mark.parametrize("chunk_size", range(100000, 500000, 100000))
    def test_coffea_processor_adlexample1(benchmark, chunk_size):
>         benchmark(coffea_processor_adlexample1, chunk_size)

<ipython-input-29-62d1ae0f702f>:3: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pytest_benchmark/fixture.py:127: in __call__
    return self._raw(function_to_benchmark, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pytest_benchmark/fixture.py:149: in _raw
    duration, iterations, loops_